In [243]:
# Import statements
# Build Dataset
# Negative Sampling
# Define Model
# Predictions & vector plotting

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import defaultdict
from tqdm.notebook import trange, tqdm

from preprocess_dataset import StanfordSentiment

# Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, path, window_size = 2, transform = None, target_transform = None):

        # Get the raw dataset
        dataset = StanfordSentiment(path)
        self.corpus = dataset.get_dataset()

        self.dataset = []
        self.window_size = window_size
        self.word2index = {}
        self.vocab = []
        self.word_freq = defaultdict(int)
        self.construct_dataset()
        
    def construct_dataset(self):
        for sentence in self.corpus:
            for i, word in enumerate(sentence):
                self.word_freq[word] += 1
                if self.word2index.get(word) is None:
                    self.word2index[word] = len(self.word2index)
                
                target_word_combinations = []
                for k in range(i - self.window_size, i + self.window_size + 1):
                    if k == i or k < 0 or k >= len(sentence):
                        continue
                    target_word_combinations.append([word] + [sentence[k]])
                self.dataset.extend(target_word_combinations)
                     
        self.index2word = {i: w for w, i in self.word2index.items()}
        self.vocab_size = len(self.index2word)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return (torch.LongTensor([self.word2index[self.dataset[index][0]]]), torch.LongTensor([self.word2index[self.dataset[index][1]]]))

In [3]:
train_dataset = CustomDataset(path = './test.txt', window_size = 2)
train_data_loader = DataLoader(train_dataset, batch_size = 1, shuffle = False)

# Negative Sampling

In [4]:
negative_sample_table_size = 1000000
negative_samples_count = 1
negative_sample_table = [0] * negative_sample_table_size

def construct_negative_sample_table(dataset):
    word_freq = [dataset.word_freq[dataset.index2word[i]] for i in range(dataset.vocab_size)]
    word_freq = np.array(word_freq) ** 0.75
    word_freq = word_freq / np.sum(word_freq)
    word_freq = np.cumsum(word_freq) * negative_sample_table_size

    j = 0
    for i in range(0, negative_sample_table_size):
        while i > word_freq[j]:
            j += 1
        negative_sample_table[i] = j

def get_sample_word():
     return negative_sample_table[random.randint(0, negative_sample_table_size - 1)]

def get_negative_samples(target_word_index, context_word_index):
    negative_samples = torch.LongTensor([])
    
    while negative_samples.shape[0] < negative_samples_count:
        sample_word_index = get_sample_word()
        while sample_word_index == target_word_index or sample_word_index == context_word_index:
            sample_word_index = get_sample_word()
        negative_samples = torch.cat((negative_samples, torch.LongTensor([sample_word_index])), dim = 0)
    return negative_samples

def generate_batch_negative_samples(target_word_indices, context_word_indices):
    batches = target_word_indices.shape[0]

    negative_samples = torch.LongTensor([])
    for i in range(batches):
        negative_sample = get_negative_samples(target_word_indices[i].data, context_word_indices[i].data)
        negative_samples = torch.cat((negative_samples, negative_sample.unsqueeze(0)), dim = 0)

    return negative_samples

In [5]:
construct_negative_sample_table(train_dataset)

# Model

In [6]:
class Model(nn.Module):
    def __init__(self, embedding_size, vocab_size):
        super().__init__()
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size

        self.hidden_layer = nn.Embedding(vocab_size, embedding_size)
        self.output_layer = nn.Embedding(vocab_size, embedding_size)
        self.logSigmoid = nn.LogSigmoid()

        self.hidden_layer.weight.data.uniform_(-1.0, 1.0)
        self.output_layer.weight.data.uniform_(-1.0, 1.0)
        
    def forward(self, target_word_indices, context_word_indices, negative_word_indices):
        target_word_embeddings = self.hidden_layer(target_word_indices) # B x 1 x D
        context_word_embeddings = self.output_layer(context_word_indices) # B x 1 x D
        negative_word_embeddings = -self.output_layer(negative_word_indices) # B x K x D
        

        context_word_scores = (context_word_embeddings.bmm(target_word_embeddings.transpose(1, 2))).squeeze(2).squeeze(1) # 1D tensor with B elements
        negative_word_scores = torch.sum((negative_word_embeddings.bmm(target_word_embeddings.transpose(1, 2))).squeeze(2), axis = 0) # 1D tensor with B elements
        
        pos_loss = self.logSigmoid(context_word_scores)
        neg_loss = self.logSigmoid(negative_word_scores)
        loss = pos_loss + neg_loss
        return -torch.mean(loss)

    def get_word_vector(self, index):
        return self.hidden_layer(index)

In [7]:
model = Model(4, train_dataset.vocab_size)

# Training 

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.005)

In [10]:
def train(model, iterator, optimizer, device):

    epoch_loss = 0

    for batch in tqdm(iterator, desc="Training", leave=False):

        target_word_indices = batch[0].to(device)
        context_word_indices = batch[1].to(device)

        negative_word_indices = generate_batch_negative_samples(target_word_indices, context_word_indices)
        

        model.zero_grad()

        loss = model(target_word_indices, context_word_indices, negative_word_indices)
        loss.backward()

        optimizer.step()

        loss = loss.item()

        epoch_loss += loss

    return epoch_loss

In [12]:
EPOCHS = 4000

for epoch in trange(EPOCHS):
    epoch_loss = train(model, train_data_loader, optimizer, device)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {epoch_loss:.3f}')

  0%|          | 0/4000 [00:00<?, ?it/s]

Training:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 01
	Train Loss: 34.421


Training:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 02
	Train Loss: 34.859


...

Epoch: 3998
	Train Loss: 15.737


Training:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 3999
	Train Loss: 14.480


Training:   0%|          | 0/48 [00:00<?, ?it/s]

Epoch: 4000
	Train Loss: 7.821


In [13]:
# returns top-10 similar words
def word_similarity(target_word, vocab):
    target_embedding = model.get_word_vector(torch.LongTensor([train_dataset.word2index[target_word]]))

    similarities = []
    for context_word in vocab:
        if context_word == target_word: 
            continue
        
        context_embedding = model.get_word_vector(torch.LongTensor([train_dataset.word2index[context_word]]))
        cosine_sim = F.cosine_similarity(target_embedding, context_embedding).data.tolist()[0]
        similarities.append([context_word, cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)

In [16]:
word_similarity('ford', train_dataset.word_freq.keys())

[['car', 0.8867323994636536],
 ['usa', 0.8235629796981812],
 ['boston', 0.7941510677337646],
 ['mercedes', 0.6681135892868042],
 ['cold', 0.6085545420646667],
 ['sugar', 0.5884252190589905],
 ['bacon', 0.4463612735271454],
 ['milk', 0.3560340404510498],
 ['drink', 0.2991504371166229],
 ['apple', 0.2857401371002197],
 ['mango', 0.280969500541687],
 ['germany', 0.27739208936691284],
 ['cherry', 0.2455643117427826],
 ['eat', 0.22984224557876587],
 ['berlin', 0.1905454695224762],
 ['fruit', 0.16131195425987244],
 ['juice', 0.13771706819534302],
 ['water', 0.05670595169067383],
 ['cola', -0.04299187660217285]]